In [1]:
import pandas as pd

In [2]:
train = pd.read_csv(r'C:\Users\bosss\Desktop\Тестовое АВИТО\train.csv')
val = pd.read_csv(r'C:\Users\bosss\Desktop\Тестовое АВИТО\val.csv')

## Обработаем валидационные данные и глянем на корреляцию между признаками и целевым признаком

In [3]:
new_val = val.drop(['title', 'datetime_submitted'], axis = 1)
# new_val

In [4]:
kategor = ['subcategory', 'category', 'region', 'city']
ne_kategor = ['description', 'price', 'is_bad']

In [5]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

In [6]:
encoder = OrdinalEncoder()
encoder.fit(new_val[kategor])
data_ordinal = encoder.transform(new_val[kategor])
data_ordinal = pd.DataFrame(encoder.transform(new_val[kategor]), columns = new_val[kategor].columns)
# data_ordinal

In [7]:
n_val = data_ordinal.join(new_val[ne_kategor])
n_val

,subcategory,category,region,city,description,price,is_bad
0,14.0,7.0,70.0,715.0,Звонить 89425546881,2000.0,1
1,30.0,1.0,11.0,196.0,"Мобильная перегородка, предназначена для разгр...",10500.0,0
2,14.0,7.0,54.0,620.0,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,4000.0,1
3,24.0,2.0,41.0,1073.0,"Продаю кровать-трансформер производства ""Столп...",17000.0,0
4,25.0,7.0,7.0,148.0,"Мот в отличном состоянии для своих лет, Родной...",235000.0,0
...,...,...,...,...,...,...,...
16232,20.0,5.0,80.0,1091.0,Сдаются офисное помещение по Ул Калинина 80. П...,9200.0,0
16233,44.0,0.0,67.0,382.0,Оригинальный ростест айфон. Идеальное состояни...,20000.0,0
16234,17.0,5.0,65.0,813.0,______________________________________________...,3300000.0,0
16235,17.0,5.0,65.0,813.0,/\n /\n● Работаем БЕЗ ПЕРЕРЫВОВ И ВЫХОДНЫХ с ...,2300000.0,0


In [8]:
# scaler = StandardScaler()
# scaler.fit(n_val)
# val_ = scaler.transform(n_val)
# val_

In [9]:
# _val_ = pd.DataFrame(val_, columns = n_val.columns)
# _val_

In [10]:
# _val_.corr()

Сильная связь отсутствует, в таком случае, попробуем для начала обучить на CatBoostClassifier с категориальными признаками, посмотрим на результат. И в зависимости от него будем двигаться дальше

## Обработаем 80тыс. тренировочных данных

In [11]:
new_train = train.drop(['title', 'datetime_submitted'], axis = 1)
# new_train

In [12]:
sample = 80000
new_train = new_train.sample(n = sample, random_state = 1234).reset_index(drop=True)

In [13]:
kategor = ['subcategory', 'category', 'region', 'city']
ne_kategor = ['description', 'price', 'is_bad']

In [14]:
encoder = OrdinalEncoder()
encoder.fit(new_train[kategor])
data_ordinal = encoder.transform(new_train[kategor])
data_ordinal = pd.DataFrame(encoder.transform(new_train[kategor]), columns = new_train[kategor].columns)
# data_ordinal

In [15]:
n_train = data_ordinal.join(new_train[ne_kategor])
n_train

,subcategory,category,region,city,description,price,is_bad
0,29.0,0.0,55.0,1329.0,В комплекте основная зарядка и док-станция WD4...,45000.0,0
1,18.0,9.0,63.0,622.0,Есть еще другая литература до с 30 до 80х годо...,NaN,0
2,19.0,9.0,55.0,1329.0,Продаю старые советские монеты времен СССР/\nц...,800.0,0
3,10.0,7.0,73.0,2167.0,Породам бурильную установку МГБУ-ТС 25 полност...,210000.0,0
4,14.0,7.0,25.0,998.0,Сигнал заднего хода/\nPreco Model 370/\n507 ДБ...,2500.0,0
...,...,...,...,...,...,...,...
79995,0.0,7.0,13.0,558.0,Машина в хорошем состоянии стоит газ форсуночн...,160000.0,1
79996,45.0,4.0,10.0,390.0,"Продаю новые селиконовые накладки для груди, р...",250.0,0
79997,11.0,4.0,55.0,1329.0,Костюм для девочки /\nРазмеры 602 и 660/\nСост...,1000.0,0
79998,2.0,0.0,84.0,1841.0,"Корпус для РЭА Gainta G0425 444х10х60, в налич...",400.0,0


In [16]:
# scaler = StandardScaler()
# scaler.fit(n_train)
# train_ = scaler.transform(n_train)
# train_

In [17]:
# _train_ = pd.DataFrame(train_, columns = n_train.columns)
# _train_

## Обучение на трейне

In [18]:
feature_train = n_train.drop(['is_bad'], axis = 1)
target_train = n_train['is_bad']

In [19]:
feature_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 6 columns):
subcategory    80000 non-null float64
category       80000 non-null float64
region         80000 non-null float64
city           80000 non-null float64
description    80000 non-null object
price          75366 non-null float64
dtypes: float64(5), object(1)
memory usage: 3.7+ MB


In [20]:
# feature_train_ = feature_train.drop(['title', 'datetime_submitted'], axis = 1)
# feature_train_

In [21]:
feature_train_ = feature_train.fillna(0)
feature_train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 6 columns):
subcategory    80000 non-null float64
category       80000 non-null float64
region         80000 non-null float64
city           80000 non-null float64
description    80000 non-null object
price          80000 non-null float64
dtypes: float64(5), object(1)
memory usage: 3.7+ MB


In [22]:
feature_train_

,subcategory,category,region,city,description,price
0,29.0,0.0,55.0,1329.0,В комплекте основная зарядка и док-станция WD4...,45000.0
1,18.0,9.0,63.0,622.0,Есть еще другая литература до с 30 до 80х годо...,0.0
2,19.0,9.0,55.0,1329.0,Продаю старые советские монеты времен СССР/\nц...,800.0
3,10.0,7.0,73.0,2167.0,Породам бурильную установку МГБУ-ТС 25 полност...,210000.0
4,14.0,7.0,25.0,998.0,Сигнал заднего хода/\nPreco Model 370/\n507 ДБ...,2500.0
...,...,...,...,...,...,...
79995,0.0,7.0,13.0,558.0,Машина в хорошем состоянии стоит газ форсуночн...,160000.0
79996,45.0,4.0,10.0,390.0,"Продаю новые селиконовые накладки для груди, р...",250.0
79997,11.0,4.0,55.0,1329.0,Костюм для девочки /\nРазмеры 602 и 660/\nСост...,1000.0
79998,2.0,0.0,84.0,1841.0,"Корпус для РЭА Gainta G0425 444х10х60, в налич...",400.0


In [23]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier

In [24]:
# model_cat = CatBoostClassifier(iterations=1000,
#                           learning_rate=0.001,
#                           verbose=False,
#                           loss_function='Logloss',
#                           text_features = ['description','subcategory','category','region','city'])

In [25]:
n_val = n_val.fillna(0)

In [27]:
train, test = train_test_split(n_val, stratify = n_val['is_bad'], test_size = 0.20, random_state = 12345)

In [28]:
val_train_features = train.drop(['is_bad'], axis = 1)
val_train_target = train['is_bad']
val_test_features = test.drop(['is_bad'], axis = 1)
val_test_target = test['is_bad']


In [29]:
model_cat = CatBoostClassifier(iterations=1000,
                          learning_rate=0.001,
                          verbose=False,
                          loss_function='Logloss',
                          text_features = [4])
                          #cat_features = ['subcategory','category','region','city'])

In [30]:
# %%time

# param = {'l2_leaf_reg': [1, 9, 20],
#         'learning_rate': [0.001, 0.01, 0.1],
#         'depth': [4, 6, 8]} #10, 12]}

# grid = model_cat.grid_search(param, X=feature_train_, y=target_train)

In [31]:
# grid.best_params_

In [32]:
# model_grid = CatBoostClassifier(iterations=2000,
#                           verbose=False,
#                           loss_function='RMSE',
#                           **grid['param'])

In [33]:
%%time

model_cat.fit(val_train_features, val_train_target)

In [34]:
# test['description'] = test['description'].astype('str')
# test.info()

In [35]:
probabilities_valid = model_cat.predict_proba(val_test_features)
proba_one = probabilities_valid[:,1]

In [36]:
proba_one

array([0.14318212, 0.64534188, 0.72958549, ..., 0.54383308, 0.11043416,
       0.13075288])

In [37]:
total_one = pd.DataFrame({'index': val_test_target.index, 'prediction': proba_one})
total_one

,index,prediction
0,1468,0.143182
1,6624,0.645342
2,1621,0.729585
3,4520,0.151078
4,14377,0.585444
...,...,...
3243,15192,0.429188
3244,4497,0.626399
3245,16194,0.543833
3246,2853,0.110434


In [38]:
from sklearn.metrics import roc_auc_score

# pred_test = model_linear_01.predict_proba(tf_idf_test)[:, 1]
print('ROC-AUC:', roc_auc_score(val_test_target, total_one['prediction']))

ROC-AUC: 0.9235067056325675
